# Nettoyage de la table T_complete
Requete SELECT test et instruction UPDATE pour mise à jour
Analyse des colonnes Pays et age

In [ ]:
import psycopg2
import os
import csv
import pandas as pd
import re

In [ ]:
def chemin_fichier()-> str :
    """
        Donne le nom du chemin complet du fichier actuel (.py) 
    """
    chemin=os.getcwd() #methode pour tourner avec jupyter
    #chemin= path.dirname(path.abspath(__file__)) #methode d'Alice avec <os>
    #chemin = str(Path(__file__).parent) #ma methode avec <path>
    #print(chemin)
    return chemin

In [ ]:
# etablir une connection
def connexion_DB(fichierDB):
    try:
        conn = psycopg2.connect(user = "santa",
                                password = "claus",
                                host = "localhost",
                                port = "5432:5432",
                                database = fichierDB
                                )

        cur = conn.cursor()
        # Print PostgreSQL Connection properties
        #print ( "Proprietes : ",conn.get_dsn_parameters(),"\n")

        # Print PostgreSQL version
        cur.execute("SELECT version();")
        record = cur.fetchone()
        #print("Version de PostgreSQL : ", record,"\n")
        cur.close()
    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)
    
    return conn

In [ ]:
def deconnexion_DB(conn):
#closing database connection.
    if(conn):
        conn.close()
        print("PostgreSQL connection is closed")

In [ ]:
conn=connexion_DB("santa_data")

path_dossier_letters=chemin_fichier() +'/Partie_1/lettres_pere_noel'

cursor = conn.cursor()

In [ ]:
# analyse des differentes occurences du champ 'age'
# pour preparer les UPDATE

rq_select='''
          SELECT
          count(ID),
          age
          FROM T_complete
          GROUP BY age
          ORDER BY age;
          '''    
cursor.execute(rq_select)
res=cursor.fetchall()

print('\nNb'.ljust(8),'age'.ljust(15))
trait="-"*40
print(trait)
for row in res:
    print(str(row[0]).ljust(8),end='')
    print(str(row[1]).ljust(15))


In [ ]:
# analyse des differentes occurences du champ 'Pays'
# pour preparer les UPDATE

rq_select='''
          SELECT
          count(ID),
          Pays
          FROM T_complete
          GROUP BY Pays
          ORDER BY Pays;
          '''    
cursor.execute(rq_select)
res=cursor.fetchall()

print('\nNb'.ljust(8),'pays'.ljust(20))
trait="-"*40
print(trait)
for row in res:
    print(str(row[0]).ljust(8),end='')
    print(str(row[1]).ljust(20))

In [ ]:
# preparation et visualisation des modifications pour harmoniser le champ 'age'
#avant les UPDATE (affiche les lignes concernees) et apres (aucune ligne)

rq_select='''
          SELECT
          ID,
          id_l,
          nom_lettre,
          nom_enfant,
          age,
          pays,
          Contenu
          FROM T_complete
          order by pays
          ;
          '''
cursor.execute(rq_select)
res=cursor.fetchall()

#print('\nid_l'.ljust(8),'nom_enfant'.ljust(25),
#    'age'.ljust(15),'Pays'.ljust(10),'Contenu'.ljust(15))
print('\nID'.ljust(8),'id_l'.ljust(8),'nom_lettre'.ljust(20),'nom_enfant'.ljust(15),
    'age'.ljust(15),'Pays'.ljust(10),'Contenu'.ljust(15))
trait="-"*90
print(trait)
for row in res:
    print(str(row[0]).ljust(8),end='')
    print(str(row[1]).ljust(8),end='')
    print(str(row[2]).ljust(20),end='')
    print(str(row[3]).ljust(15),end='')
    print(str(row[4]).ljust(15),end='')
    print(str(row[5]).ljust(10),end='')
    print(str(row[6])[0:30].ljust(15))
        
            

ID,id_l,nom_lettre,nom_enfant,age,Pays,Contenu,Numero

In [ ]:
#traitement des pays où ''  mis a 'None'
cursor.execute('''
                UPDATE T_complete 
                SET Pays=NULL
                WHERE Pays='';
               ''')
conn.commit()

In [ ]:
#traitement des pays où '5 ans' et 1 2éme prenom dans age (ID=704 et 750)
cursor.execute('''
                UPDATE T_complete 
                SET nom_enfant=nom_enfant || ',' || age
                WHERE Pays='5 ans';
               ''')
conn.commit()

In [ ]:
#traitement des pays où '5 ans'
cursor.execute('''UPDATE T_complete 
                SET age=Pays
                WHERE ID=704 OR ID=750;
               ''')
conn.commit()

In [ ]:
#traitement des pays où '5 ans'
cursor.execute('''UPDATE T_complete 
                SET Pays = NULL
                WHERE ID=704 OR ID=750;
               ''')
conn.commit()

In [ ]:
#age='France' mis dans Pays
cursor.execute('''UPDATE T_complete 
                SET Pays = age
                WHERE age='France';
               ''')
conn.commit()

In [ ]:
#age='France' mis A None
cursor.execute('''UPDATE T_complete 
                SET age = NULL
                WHERE age='France';
               ''')
conn.commit()

In [ ]:
#age= '1 pays' : age mis dans Pays ; et age mis à None
cursor.execute('''UPDATE T_complete 
                SET Pays = age, age = NULL
                WHERE age IN (SELECT DISTINCT Pays FROM T_COMPLETE)
          ;
               ''')
conn.commit()

In [ ]:
#age exprimé en mois, traduit en année, et mois remplacé par ans
cursor.execute('''UPDATE T_complete 
                SET age = trunc(cast(trim(trailing ' mois et' from age) as numeric)/12 ,1) || ' ans'
                WHERE age SIMILAR TO '%mois%'
          ;
               ''')
conn.commit()

In [ ]:
# demi age  remplace par .5
cursor.execute('''UPDATE T_complete 
                SET age = regexp_replace(age, 'et demi| et demi|  et demi', '.5') 
                WHERE age SIMILAR TO '%et demi%'
          ;
               ''')
conn.commit()

In [ ]:
#a# 1/2 age  remplace par .5 (\s+  = 1 espace ou +)
cursor.execute('''UPDATE T_complete 
                SET age = regexp_replace(age, '(1/2|\s+1/2| et 1/2|, 1/2|an 1/2| ans 1/2)', '.5')
                WHERE age SIMILAR TO '%1/2%'
          ;
               ''')
conn.commit()

In [ ]:
# ½ age  remplace par .5 (\s+  = 1 espace ou +)
cursor.execute('''UPDATE T_complete 
                SET age = regexp_replace(age, '(½| ½)', '.5')
                WHERE age SIMILAR TO '%½%'
          ;
               ''')
conn.commit()

In [ ]:
# an et dem  remplace par .5 (\s+  = 1 espace ou +)
cursor.execute('''UPDATE T_complete 
                SET age = regexp_replace(age, 'an et dem', '.5')
                WHERE age SIMILAR TO '%an et dem%'
          ;
               ''')
conn.commit()

In [ ]:
# et d. remplace par .5 (\s+  = 1 espace ou +)
cursor.execute('''UPDATE T_complete 
                SET age = regexp_replace(age, '\s+et d|\s+et de', '.5')
                WHERE age SIMILAR TO '%et d%'
          ;
               ''')
conn.commit()

In [ ]:
# 08 remplace par 8
cursor.execute('''UPDATE T_complete 
                SET age = regexp_replace(age, '08', '8')
                WHERE age SIMILAR TO '%08%'
          ;
               ''')
conn.commit()

In [ ]:
# , remplace par .
cursor.execute('''UPDATE T_complete 
                SET age = regexp_replace(age, ',', '.')
                WHERE age SIMILAR TO '%,5 ans%'
          ;
               ''')
conn.commit()

In [ ]:
# , remplace par .
cursor.execute('''UPDATE T_complete 
                SET age = regexp_replace(age, 'edemie', '.5')
                WHERE age SIMILAR TO '%edemie%'
          ;
               ''')
conn.commit()


In [ ]:
# ' ' supprime en fin de chaine .
cursor.execute('''UPDATE T_complete 
                SET age = TRIM(TRAILING ' ' FROM age)
                WHERE age SIMILAR TO '%ans %'
          ;
               ''')
conn.commit()

In [ ]:
# supprime la repetition de ans .
cursor.execute('''UPDATE T_complete 
                SET age = regexp_replace(age, 'ans ans', 'ans')
                WHERE age SIMILAR TO '%ans ans%'
          ;
               ''')
conn.commit()


In [ ]:
# supprime '1 '
cursor.execute('''UPDATE T_complete 
                SET age = regexp_replace(age, '2   1 ans', '2 ans')
                WHERE age SIMILAR TO '%2   1 ans%'
          ;
               ''')
conn.commit()



In [ ]:
# supprime 'a '
cursor.execute('''UPDATE T_complete 
                SET age = regexp_replace(age, '2 a ans', '2 ans')
                WHERE age SIMILAR TO '%2 a ans%'
          ;
               ''')
conn.commit()

In [ ]:
# supprime 'ann '
cursor.execute('''UPDATE T_complete 
                SET age = regexp_replace(age, 'ann ', '')
                WHERE age SIMILAR TO '%ann %'
          ;
               ''')
conn.commit()

In [ ]:
# change 3/4 en .7
cursor.execute('''UPDATE T_complete 
                SET age = regexp_replace(age, '8 3/4 ans', '8.7 ans')
                WHERE age SIMILAR TO '%8 3/4 ans%'
          ;
               ''')
conn.commit()

In [ ]:
# change ben j'en a ans|j'ai en ''
cursor.execute('''UPDATE T_complete 
                SET age = regexp_replace(age, 'ben j''en a ans|j''ai' , '')
                WHERE age SIMILAR TO '%j''%'
          ;
               ''')
conn.commit()


In [ ]:
#age='' mis A None
cursor.execute('''UPDATE T_complete 
                SET age = NULL
                WHERE age='None' or age='';
               ''')
conn.commit()

In [ ]:
# change ben j'en a ans|j'ai en ''
cursor.execute('''UPDATE T_complete 
                SET age = regexp_replace(age, 'onze' , '11')
                WHERE age SIMILAR TO '%onze%'
          ;
               ''')
conn.commit()


In [ ]:
# change '2 et4mo ans' en '2.4ans'
cursor.execute('''UPDATE T_complete 
                SET age = regexp_replace(age, '2 et4mo', '2.4')
                WHERE age SIMILAR TO '%2 et4mo ans%' 
          ;
               ''')
conn.commit()

In [ ]:
# change '5/5 ans' en '5.5 ans'
cursor.execute('''UPDATE T_complete 
                SET age = regexp_replace(age, '/', '.')
                WHERE age SIMILAR TO '%(5/5)%' 
          ;
               ''')
conn.commit()

In [ ]:
# change age1/age2 ans' en 'age2 ans'
cursor.execute('''UPDATE T_complete 
                SET age = regexp_replace(age,'[0-9]/','')
                WHERE age SIMILAR TO '%/%' 
          ;
               ''')
conn.commit()

In [ ]:
# change ' 12 ans' en '12 ans'
cursor.execute('''UPDATE T_complete 
                SET age = regexp_replace(age, ' 12 ans','12 ans')
                WHERE age SIMILAR TO '% 12 ans%'
          ;
               ''')
conn.commit()

In [ ]:
# change ' 7ou8ou9 bi' en '8 ans'
cursor.execute('''UPDATE T_complete 
                SET age = regexp_replace(age, '7ou8ou9 bi','8')
                WHERE age SIMILAR TO '%7ou8ou9 bi%'
          ;
               ''')
conn.commit()

In [ ]:
# change nom_enfant' Charlie...euh Charlène' en 'Charlène'
cursor.execute('''UPDATE T_complete 
                SET nom_enfant = 'Charlène'
                WHERE nom_enfant = 'Charlie...euh Charlène'
          ;
               ''')
conn.commit()

In [ ]:
cursor.close()
deconnexion_DB(conn)